In [10]:
import numpy as np
import pandas as pd
import re

In [11]:
df_train = pd.read_csv('train.csv')

df_test = pd.read_csv('test.csv')
df_test.head()

,ID,Comment
0,0,bhai ko kiska paper ha
1,1,thank me later
2,2,thankqqqqq mam for this video
3,3,apki teaching se physics sahi main ek visuliza...
4,4,i didnt understand the differentiation part co...


In [12]:
y_train = df_train['Label']
X_train = df_train['Comment'].str.lower()

X_test = df_test['Comment'].str.lower()
y_test = pd.read_csv('sample_submission.csv')
y_test = y_test['Label']

In [13]:
# Remove url's
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

for i in range(X_train.shape[0]):
    X_train[i]=remove_url(X_train[i])
    
for j in range(X_test.shape[0]):
    X_test[j] = remove_url(X_test[j])

In [14]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

#Replace chat words
def chat_conversion(text):
    new_text = []
    for i in text.split():
        if i.upper() in chat_words:
            new_text.append(chat_words[i.upper()])
        else:
            new_text.append(i)
    return " ".join(new_text)

#Remove extra white spaces
def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

for i in range(X_train.shape[0]):
    X_train[i]  = chat_conversion(X_train[i])
    X_train[i] = remove_extra_spaces(X_train[i])
    
for i in range(X_test.shape[0]):
    X_test[i]  = chat_conversion(X_test[i])
    X_test[i] = remove_extra_spaces(X_test[i])
    

    

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=5000, min_df=2)


X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)


In [16]:
# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(max_iter= 1000)
# model.fit(X_train, y_train)

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga'],
    'max_iter': [500, 1000]
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [17]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average='macro')
print("F1 Score:", f1)


F1 Score: 0.3259248497855746


In [18]:
print(y_pred)

['irrelevant' 'irrelevant' 'feedback' ... 'feedback' 'feedback'
 'irrelevant']


In [19]:
y = pd.read_csv('test.csv')
submission = pd.DataFrame({
    'ID': y['ID'],
    'Label': y_pred
})

submission.to_csv('submission.csv', index=False)

print("\nPreview of the submission file:")
print(submission.head())


Preview of the submission file:
   ID       Label
0   0  irrelevant
1   1  irrelevant
2   2    feedback
3   3    feedback
4   4       doubt
